#Konfiguracja i integracja z Dyskiem Google

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

#Konfiguracja Fast.AI

In [0]:
!git clone https://github.com/fastai/fastai.git
import os
os.chdir("fastai")
!git pull

In [0]:
os.chdir("..")
!pip install https://github.com/fastai/fastai/archive/master.zip

# Konfiguracja PyTorch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

# ipywidgets fix

In [0]:
!pip install ipywidgets==6.0.0

In [0]:
#Now go to your Google Drive and open the lesson1 notebook
#Make sure you have runtime type for this as well as the notebook in google drive to GPU

# Instalacja PyDrive wrapper & import bibliotek

In [0]:
# This only needs to be done once per notebook.
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from pathlib import Path
import os

def download_and_save(file_name, file_id):
  downloaded = drive.CreateFile({'id': file_id})
  # fetch file
  downloaded.FetchContent()
  # write file
  with open(file_name,'wb') as f:
       f.write(downloaded.content.read())
      
  print(f'Saved {file_name}')

# Uzyskiwanie zestawu danych

In [0]:
dataset_path = Path.home() / 'data/old_polish_cars'; dataset_path

In [0]:
!mkdir -p $dataset_path

In [0]:
cd $dataset_path

In [0]:
ls -la $dataset_path/

## Uzyskiwanie old_polish_cars_two_classes_v2a-split dataset
https://drive.google.com/file/d/1L0XBjL5fFE41OrrahDBWbDrICK2OpiMC/view?usp=sharing

In [0]:
file_name = dataset_path / 'old_polish_cars_two_classes_v2a-split.zip'
file_id = '1L0XBjL5fFE41OrrahDBWbDrICK2OpiMC'

download_and_save(file_name, file_id)

In [0]:
cd $dataset_path

In [0]:
!unzip -q -o $file_name

In [0]:
cd ~

## Pobieranie przykładowego modelu danych dla inferencji
https://drive.google.com/file/d/1APQQCeDAqMCOpjoNgmWTnBJFykifCS6w/view?usp=sharing

In [0]:
file_folder = dataset_path / 'old_polish_cars_two_classes_v2a-split/models/'
file_name = file_folder / '224_epoch9.h5'

In [0]:
!mkdir -p $file_folder

In [0]:
file_id = '1APQQCeDAqMCOpjoNgmWTnBJFykifCS6w'

download_and_save(file_name, file_id)

## Pobieranie folderu test2 dla eksperymentu z inferencją
https://drive.google.com/file/d/1Ki7n3xQ1mNsh19hi9k7VS5LQkynMk24p/view?usp=sharing

In [0]:
file_folder = dataset_path / 'old_polish_cars_two_classes_v2a-split'
file_name = file_folder / 'test2.zip'
file_id = '1Ki7n3xQ1mNsh19hi9k7VS5LQkynMk24p'

download_and_save(file_name, file_id)

In [0]:
cd $file_folder

In [0]:
!unzip -q -o $file_name

In [0]:
cd ~

# Nadzorowanie wirtualnej maszyny

In [0]:
!pwd

In [0]:
!df -h

In [0]:
!ls -lah ~/data/old_polish_cars/old_polish_cars_two_classes_v2a-split/test2

In [0]:
who

In [0]:
!uptime

In [0]:
!/opt/bin/nvidia-smi

In [0]:
!top -n 1

In [0]:
import os
import psutil

def print_memsize():
  process = psutil.Process(os.getpid())
  print(f'{process.memory_info().rss / 1024**3:.5} GB')

In [0]:
print_memsize()

In [0]:
!ps ax|grep python